## I. 패키지 밎 자료 불러오기

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient

     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 87 kB 3.4 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99970 sha256=65de6aeb0cb149d73f19222d009f797019892c48a35bede44b1500bd925430e2
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 135 kB 5.2 MB/s 
     |████████████████████████████████| 130 kB 38.4 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM cj'
cursor.execute(sql)
result = cursor.fetchall()
DF = pd.DataFrame(result)


In [ ]:
DF.head(110)

,index,date,coin_name,s,s.1,lang
0,0,2021-11-17,ada,2,97792,eng
1,1,2021-11-17,ada,0,37535,eng
2,2,2021-11-17,ada,1,124,eng
3,3,2021-11-17,bch,2,152,eng
4,4,2021-11-17,bch,0,67,eng
...,...,...,...,...,...,...
105,18,2021-11-25,xrp,2,726,eng
106,19,2021-11-25,xrp,0,337,eng
107,0,2016-05-31,bch,0,24,kor
108,1,2016-05-31,bch,1,4,kor


In [ ]:
DF.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_db_2.csv')

## I. 일단 긍정/중립/부정 3개의 데이터 프래임으로 모으기
- 없을시 0으로 해야 함으로 데이터 프레임을 먼저 만들고 거기에 합치기
- s.1은 갯수

In [ ]:
date = ['2021-11-17','2021-11-18','2021-11-19','2021-11-20','2021-11-21','2021-11-22',
        '2021-11-23','2021-11-24','2021-11-25','2021-11-26']
        # ,'2021-11-27','2021-11-28','2021-11-29','2021-11-30']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 새로운 데이터 프레임을 만들어야한다.

In [ ]:
# 새로운 데이터 프레임 만들기
df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
for i in date:
  for j in crpyto_name:
    df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
df

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
0,2021-11-17,ada,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-11-17,bch,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-11-17,btc,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-11-17,doge,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-11-17,eos,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
85,2021-11-26,eos,NaN,NaN,NaN,NaN,NaN,NaN
86,2021-11-26,eth,NaN,NaN,NaN,NaN,NaN,NaN
87,2021-11-26,omg,NaN,NaN,NaN,NaN,NaN,NaN
88,2021-11-26,snt,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# # 연습 빈공간 지정해서 넣기
# for i in range(len(DF)):
#   if (DF['date'][i] == '2021-11-22') and (DF['coin_name'][i] == 'ada') and(DF['lang'][i] == 'kor'):
#     if (DF['s'][i] == 0):
#       print(DF['s.1'][i])
#       a = df.index[(df['date'] == '2021-11-22')&(df['crypto_name'] == 'ada')].tolist()
#       df['positive_K'][a] = DF['s.1'][i]
#     else:
#       print('없음')

### 2) 확장 - 모든 코인 모든 날짜

In [ ]:
# for j in date:

#   for i in range(len(DF)):


#     if (DF['date'][i] == j) and (DF['coin_name'][i] == 'ada') and (DF['lang'][i] == 'kor'):
#       if (DF['s'][i] == 0):
#         a = df.index[(df['date'] == j)&(df['crypto_name'] == 'ada')&(DF['lang'][i] == 'kor')].tolist()
#         df['positive_K'][a] = DF['s.1'][i]

In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF)):


      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['positive_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 0):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['positive_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['neutrual_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 1):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['neutrual_E'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
          df['negative_K'][a] = DF['s.1'][i]

      if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
        if (DF['s'][i] == 2):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
          df['negative_E'][a] = DF['s.1'][i]    

------ 해킹당한 날 빽업 파일 붙여넣기

In [ ]:
db = pymysql.connect(host=, charset='utf8')
cursor = db.cursor(pymysql.cursors.DictCursor)
sql = 'SELECT * FROM graphing_backup_Nov23'
cursor.execute(sql)
result = cursor.fetchall()
DF_backup = pd.DataFrame(result)


In [ ]:
for k in crpyto_name:

  for j in date:

    for i in range(len(DF_backup)):


      if (DF_backup['date'][i] == j) and (DF_backup['crypto_name'][i] == k):
          a = df.index[(df['date'] == j)&(df['crypto_name'] == k)].tolist()

          df['positive_E'][a] = DF_backup['positive_E'][i]
          df['neutrual_E'][a] = DF_backup['neutrual_E'][i]
          df['negative_E'][a] = DF_backup['negative_E'][i]
          df['positive_K'][a] = DF_backup['positive_K'][i]
          df['neutrual_K'][a] = DF_backup['neutrual_K'][i]
          df['negative_K'][a] = DF_backup['negative_K'][i]


----- 여기까지

In [ ]:
df = df.fillna(0)

In [ ]:
df.tail(20)

,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K
70,2021-11-24,snt,27,0,101,0,0,1
71,2021-11-24,xrp,2986,11,7441,35,8,21
72,2021-11-25,ada,7542,18,16138,7,3,9
73,2021-11-25,bch,21,0,44,10,4,3
74,2021-11-25,btc,727,1,1461,1,0,0
75,2021-11-25,doge,787,0,1429,7,7,7
76,2021-11-25,eos,43,0,66,26,16,21
77,2021-11-25,eth,2720,9,5001,2,0,0
78,2021-11-25,omg,0,0,2,0,0,0
79,2021-11-25,snt,2,0,6,2,0,0


## --------- 전처리 작업 완료

In [ ]:
# 저장포인트
df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

In [ ]:
# 저장포인트
daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
daily_conn = daily_engine.connect()
df.to_sql(name='df',con=daily_conn,if_exists='replace')
print('업데이트 완료 : df', '\n' )

업데이트 완료 : df 



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

## -------- 코랩에서 저장 포인트

## II. 지수화
- 일단 긍정의 전체비율만을 가지고 한다. (대부분의 경우 상승에 관심 있고 롱 포지션을 가지기때문에)
- 영어가 대부분이고 영어는 거의 중립이 없어서 가능할지도
- 나중에 부정의 비율도 고려한다.


In [ ]:
date = ['2021-11-17','2021-11-18','2021-11-19','2021-11-20','2021-11-21','2021-11-22',
        '2021-11-23','2021-11-24','2021-11-25']
        # '2021-11-26','2021-11-27','2021-11-28','2021-11-29','2021-11-30']
# lang = ['kor','eng']
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
s = [0,1,2]

### 1) 긍정비율

In [ ]:
df['positive_ratio_E'] = df['positive_E']/(df['positive_E']+df['neutrual_E']+df['negative_E'])
df['positive_ratio_K'] = df['positive_K']/(df['positive_K']+df['neutrual_K']+df['negative_K'])

In [ ]:
df.head(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,0,2021-11-17,ada,37535,124,97792,6,7,3,0.277111,0.375000
1,1,2021-11-17,bch,67,0,152,20,5,10,0.305936,0.571429
2,2,2021-11-17,btc,5040,17,12443,3,2,1,0.288000,0.500000
3,3,2021-11-17,doge,2125,11,5339,10,4,4,0.284281,0.555556
4,4,2021-11-17,eos,304,2,662,13,8,17,0.314050,0.342105
5,5,2021-11-17,eth,14031,45,36835,0,0,0,0.275599,NaN
6,6,2021-11-17,omg,13,0,26,0,0,1,0.333333,0.000000
7,7,2021-11-17,snt,34,0,58,0,1,1,0.369565,0.000000
8,8,2021-11-17,xrp,1593,3,4019,13,2,10,0.283704,0.520000
9,9,2021-11-18,ada,0,0,0,1,1,2,NaN,0.250000


In [ ]:
df.tail(30)

,Unnamed: 0,date,crypto_name,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
51,51,2021-11-22,omg,0,0,4,2,0,0,0.000000,1.000000
52,52,2021-11-22,snt,14,0,2,3,0,4,0.875000,0.428571
53,53,2021-11-22,xrp,40,0,112,18,2,19,0.263158,0.461538
54,54,2021-11-23,ada,4727,8,22245,3,2,2,0.175204,0.428571
55,55,2021-11-23,bch,18,0,39,11,2,0,0.315789,0.846154
56,56,2021-11-23,btc,1806,9,1806,2,1,0,0.498757,0.666667
57,57,2021-11-23,doge,1270,8,1825,8,5,7,0.409281,0.400000
58,58,2021-11-23,eos,60,0,85,14,5,11,0.413793,0.466667
59,59,2021-11-23,eth,2959,15,7471,1,0,1,0.283293,0.500000
60,60,2021-11-23,omg,0,0,4,0,0,0,0.000000,NaN


## III. 암호화폐별 추출 밎 가격테이블 연결

### 1) 가격데이터, 감정데이터 불러오기 각각 분리

In [ ]:
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for i in crypto_name_p:
    db = pymysql.connect(host=, charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM {0}'.format(i)
    cursor.execute(sql)
    result = cursor.fetchall()
    globals()[f'{i}'] = pd.DataFrame(result)



crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for j1,j2,z1 in zip(crypto_p,crypto_p,crypto_name_p):

  X = []

  for i in range(len(j2)):
    date_time_obj = datetime.strftime(j2['index'][i], '%Y-%m-%d')
    a = date_time_obj
    b = j2['close'][i]
    # c = j2['value'][i]
    d = j2['volume'][i]
    e = 0
    f = 0
    g = 0
    h = 0
    i = 0
    j = 0
    k = 0
    l = 0

    x = [a,b,d,e,f,g,h,i,j,k,l]
    X.append(x)
  globals()['{0}'.format(z1)] = pd.DataFrame(X)
  globals()['{0}'.format(z1)].columns = ['date','close_KRW','volume','positive_E','neutrual_E','negative_E',	'positive_K',	'neutrual_K',	'negative_K','positive_ratio_E','positive_ratio_K']

In [ ]:
XRP

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,2017-09-25,203.0,2.130000e-02,0,0,0,0,0,0,0,0
1,2017-09-26,211.0,5.160000e-02,0,0,0,0,0,0,0,0
2,2017-09-27,232.0,5.040000e-02,0,0,0,0,0,0,0,0
3,2017-09-28,222.0,4.980000e-02,0,0,0,0,0,0,0,0
4,2017-09-29,220.0,5.230000e-02,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1516,2021-11-22,1300.0,2.236079e+08,0,0,0,0,0,0,0,0
1517,2021-11-23,1330.0,2.296130e+08,0,0,0,0,0,0,0,0
1518,2021-11-24,1295.0,2.780391e+08,0,0,0,0,0,0,0,0
1519,2021-11-25,1290.0,1.190549e+08,0,0,0,0,0,0,0,0


### 2) 가격데이터에 감정데이터 붙여 넣기
- 가격데이터가 더많이 있어서

In [ ]:
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']

for k,l in zip(crypto_p,crpyto_name):

  for j in date:

      a = df.index[(df['date'] == j)&(df['crypto_name'] == l)].tolist()
      b = k.index[(k['date'] == j)].tolist()
      k['positive_E'][b] = df['positive_E'][a]
      k['neutrual_E'][b] = df['neutrual_E'][a]
      k['negative_E'][b] = df['negative_E'][a]
      k['positive_K'][b] = df['positive_K'][a]
      k['neutrual_K'][b] = df['neutrual_K'][a]
      k['negative_K'][b] = df['negative_K'][a]
      k['positive_ratio_E'][b] = df['positive_ratio_E'][a]
      k['positive_ratio_K'][b] = df['positive_ratio_K'][a]

In [ ]:
BTC.tail(20)

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1504,2021-11-07,76536000.0,4447.009866,0,0,0,0,0,0,0.000000,0.000000
1505,2021-11-08,81403000.0,9132.136668,0,0,0,0,0,0,0.000000,0.000000
1506,2021-11-09,80857000.0,9216.260721,0,0,0,0,0,0,0.000000,0.000000
1507,2021-11-10,79660000.0,13173.316256,0,0,0,0,0,0,0.000000,0.000000
1508,2021-11-11,78917000.0,7167.446830,0,0,0,0,0,0,0.000000,0.000000
1509,2021-11-12,78165000.0,7117.165348,0,0,0,0,0,0,0.000000,0.000000
1510,2021-11-13,77867000.0,4350.104533,0,0,0,0,0,0,0.000000,0.000000
1511,2021-11-14,79310000.0,3826.416588,0,0,0,0,0,0,0.000000,0.000000
1512,2021-11-15,77602000.0,6972.545226,0,0,0,0,0,0,0.000000,0.000000
1513,2021-11-16,73897000.0,12978.656081,0,0,0,0,0,0,0.000000,0.000000


## ------------ 중간저장

In [ ]:
# 중간저장
ADA.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

In [ ]:
# 중간저장 backup
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  df.to_sql(name=f'{i}_back_up',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}_back_up', '\n' )

업데이트 완료 : ADA_back_up 

업데이트 완료 : BTC_back_up 

업데이트 완료 : BCH_back_up 

업데이트 완료 : DOGE_back_up 

업데이트 완료 : EOS_back_up 

업데이트 완료 : ETH_back_up 

업데이트 완료 : OMG_back_up 

업데이트 완료 : SNT_back_up 

업데이트 완료 : XRP_back_up 



In [ ]:
# 중간저장
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

for i in crypto_name_p:
  daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
  daily_conn = daily_engine.connect()
  df.to_sql(name=f'{i}',con=daily_conn,if_exists='replace')
  print('업데이트 완료 :', f'{i}', '\n' )

업데이트 완료 : ADA 

업데이트 완료 : BTC 

업데이트 완료 : BCH 

업데이트 완료 : DOGE 

업데이트 완료 : EOS 

업데이트 완료 : ETH 

업데이트 완료 : OMG 

업데이트 완료 : SNT 

업데이트 완료 : XRP 



In [ ]:
# 불러오기
ADA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ADA.csv')
BCH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BCH.csv')
BTC = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/BTC.csv')
DOGE = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/DOGE.csv')
EOS = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/EOS.csv')
ETH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/ETH.csv')
OMG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/OMG.csv')
SNT = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/SNT.csv')
XRP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/crypto_graphing/XRP.csv')

## IV. 필요데이터만 남기기
-현재는 긍정비율만

## V. 시각화
- 문제점 스케일이 다르다.

In [ ]:
ADA

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
0,2017-10-06,21.3,3.330000e-02,0,0,0,0,0,0,0.000000,0.000000
1,2017-10-07,23.7,5.200000e-02,0,0,0,0,0,0,0.000000,0.000000
2,2017-10-08,23.2,5.200000e-02,0,0,0,0,0,0,0.000000,0.000000
3,2017-10-09,25.3,5.190000e-02,0,0,0,0,0,0,0.000000,0.000000
4,2017-10-10,24.7,5.140000e-02,0,0,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1505,2021-11-22,2230.0,2.351576e+07,741,1,1692,9,7,0,0.304437,0.562500
1506,2021-11-23,2190.0,3.456718e+07,4727,8,22245,3,2,2,0.175204,0.428571
1507,2021-11-24,2095.0,1.026309e+08,62807,261,154246,12,4,9,0.289015,0.480000
1508,2021-11-25,2070.0,2.482044e+07,7542,18,16138,7,3,9,0.318255,0.368421


In [ ]:
# ADA2 = ADA.replace(0,'')

In [ ]:
ADA[1499:]

,date,close_KRW,volume,positive_E,neutrual_E,negative_E,positive_K,neutrual_K,negative_K,positive_ratio_E,positive_ratio_K
1499,2021-11-16,2310.0,4.825794e+07,0,0,0,0,0,0,0.000000,0.000000
1500,2021-11-17,2300.0,3.939187e+07,37535,124,97792,6,7,3,0.277111,0.375000
1501,2021-11-18,2215.0,4.047463e+07,0,0,0,1,1,2,NaN,0.250000
1502,2021-11-19,2300.0,2.526605e+07,38911,95,108164,3,0,1,0.264395,0.750000
1503,2021-11-20,2375.0,2.302514e+07,0,0,0,11,6,5,NaN,0.500000
1504,2021-11-21,2295.0,2.164858e+07,0,0,0,5,2,1,NaN,0.625000
1505,2021-11-22,2230.0,2.351576e+07,741,1,1692,9,7,0,0.304437,0.562500
1506,2021-11-23,2190.0,3.456718e+07,4727,8,22245,3,2,2,0.175204,0.428571
1507,2021-11-24,2095.0,1.026309e+08,62807,261,154246,12,4,9,0.289015,0.480000
1508,2021-11-25,2070.0,2.482044e+07,7542,18,16138,7,3,9,0.318255,0.368421


In [ ]:
fig = px.line(ADA[1499:], x="date", y="positive_ratio_E" )
fig.show()

In [ ]:
ADA =ADA.replace(0,'')

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ADA['date'][:],y=ADA['close_KRW'][:],name='KRW',text=ADA['close_KRW'][:],yaxis='y'))
fig.add_trace(go.Scatter(x=ADA['date'][:],y=ADA['positive_ratio_E'][:],name='positive_ratio_E',text =ADA['positive_ratio_E'][:],yaxis='y2'))
fig.add_trace(go.Scatter(x=ADA['date'][:],y=ADA['positive_ratio_K'][:],name='positive_ratio_K',text =ADA['positive_ratio_K'][:],yaxis='y3'))

fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 1},
    marker={"size": 1},
    mode="lines+markers",
    showlegend=False
)


fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2021-10-31", "2021-11-25"],
        rangeslider=dict(
            autorange=True,
            range=["2021-10-31", "2021-11-25"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0.35, 1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        title="KRW",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False

    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.15],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        title="EN",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.16, 0.31],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        title="KR",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    )
)

fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)


# fig.write_html('ADA.html')

In [ ]:
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

In [ ]:
crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

for i,j in zip(crypto_name_p,crypto_p):
  
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=j['date'][-10:],y=j['close_KRW'][-10:],name='KRW',text=j['close_KRW'][-10:],yaxis='y'))
    fig.add_trace(go.Scatter(x=j['date'][-10:],y=j['positive_ratio_E'][-10:],name='positive_ratio_E',text =j['positive_ratio_E'][-10:],yaxis='y2'))
    fig.add_trace(go.Scatter(x=j['date'][-10:],y=j['positive_ratio_K'][-10:],name='positive_ratio_K',text =j['positive_ratio_K'][-10:],yaxis='y3'))

    fig.update_traces(
        hoverinfo="name+x+text",
        line={"width": 1},
        marker={"size": 3},
        mode="lines+markers",
        showlegend=False
    )


    fig.update_layout(
        xaxis=dict(
            autorange=True,
            range=["2021-10-31", "2021-11-25"],
            rangeslider=dict(
                autorange=True,
                range=["2021-10-31", "2021-11-25"]
            ),
            type="date"
        ),
        yaxis=dict(
            anchor="x",
            autorange=True,
            domain=[0.35, 1],
            linecolor="#673ab7",
            mirror=True,
            range=[-60.0858369099, 1000],
            showline=True,
            side="left",
            tickfont={"color": "#673ab7"},
            tickmode="auto",
            ticks="",
            title="KRW",
            titlefont={"color": "#673ab7"},
            type="linear",
            zeroline=False

        ),
        yaxis2=dict(
            anchor="x",
            autorange=True,
            domain=[0, 0.15],
            linecolor="#E91E63",
            mirror=True,
            range=[29.3787777032, 100000],
            showline=True,
            side="right",
            tickfont={"color": "#E91E63"},
            tickmode="auto",
            ticks="",
            title="EN",
            titlefont={"color": "#E91E63"},
            type="linear",
            zeroline=False
        ),
        yaxis3=dict(
            anchor="x",
            autorange=True,
            domain=[0.16, 0.31],
            linecolor="#E91E63",
            mirror=True,
            range=[29.3787777032, 100000],
            showline=True,
            side="right",
            tickfont={"color": "#E91E63"},
            tickmode="auto",
            ticks="",
            title="KR",
            titlefont={"color": "#E91E63"},
            type="linear",
            zeroline=False
        )
    )

    fig.update_layout(
        dragmode="zoom",
        hovermode="x",
        legend=dict(traceorder="reversed"),
        height=600,
        template="plotly_white",
        margin=dict(
            t=100,
            b=100
        ),
    )


    fig.write_html('{0}.html'.format(i))

In [ ]:
# Create figure
fig = go.Figure()

# Add traces
# 이름 설정
fig.add_trace(go.Scatter(
    x=["2013-01-15", "2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02",
       "2013-08-27",
       "2013-10-22", "2014-01-20", "2014-05-05", "2014-07-01", "2015-02-09",
       "2015-04-13",
       "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"],
    y=["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9",
       "14"],
    name="var0",
    text=["8", "3", "2", "10", "5", "5", "6", "8", "3", "3", "7", "5", "10", "10", "9",
          "14"],
    yaxis="y",
))

fig.add_trace(go.Scatter(
    x=["2015-04-13", "2015-05-13", "2015-06-08", "2015-08-05", "2016-02-25"],
    y=["53.0", "69.0", "89.0", "41.0", "41.0"],
    name="var1",
    text=["53.0", "69.0", "89.0", "41.0", "41.0"],
    yaxis="y2",
))

fig.add_trace(go.Scatter(
    x=["2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02", "2013-08-27",
       "2013-10-22",
       "2014-01-20", "2014-04-09", "2014-05-05", "2014-07-01", "2014-09-30",
       "2015-02-09",
       "2015-04-13", "2015-06-08", "2016-02-25"],
    y=["9.6", "4.6", "2.7", "8.3", "18", "7.3", "3", "7.5", "1.0", "0.5", "2.8", "9.2",
       "13", "5.8", "6.9"],
    name="var2",
    text=["9.6", "4.6", "2.7", "8.3", "18", "7.3", "3", "7.5", "1.0", "0.5", "2.8",
          "9.2",
          "13", "5.8", "6.9"],
    yaxis="y3",
))

fig.add_trace(go.Scatter(
    x=["2013-01-29", "2013-02-26", "2013-04-19", "2013-07-02", "2013-08-27",
       "2013-10-22",
       "2014-01-20", "2014-04-09", "2014-05-05", "2014-07-01", "2014-09-30",
       "2015-02-09",
       "2015-04-13", "2015-06-08", "2016-02-25"],
    y=["6.9", "7.5", "7.3", "7.3", "6.9", "7.1", "8", "7.8", "7.4", "7.9", "7.9", "7.6",
       "7.2", "7.2", "8.0"],
    name="var3",
    text=["6.9", "7.5", "7.3", "7.3", "6.9", "7.1", "8", "7.8", "7.4", "7.9", "7.9",
          "7.6",
          "7.2", "7.2", "8.0"],
    yaxis="y4",
))

fig.add_trace(go.Scatter(
    x=["2013-02-26", "2013-07-02", "2013-09-26", "2013-10-22", "2013-12-04",
       "2014-01-02",
       "2014-01-20", "2014-05-05", "2014-07-01", "2015-02-09", "2015-05-05"],
    y=["290", "1078", "263", "407", "660", "740", "33", "374", "95", "734", "3000"],
    name="var4",
    text=["290", "1078", "263", "407", "660", "740", "33", "374", "95", "734", "3000"],
    yaxis="y5",
))

# style all the traces
# 선의 굵기등
fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 0.5},
    marker={"size": 8},
    mode="lines+markers",
    showlegend=False
)

# Add annotations
fig.update_layout(
    annotations=[
        dict(
            x="2013-06-01",
            y=0,
            arrowcolor="rgba(63, 81, 181, 0.2)",
            arrowsize=0.3,
            ax=0,
            ay=30,
            text="state1",
            xref="x",
            yanchor="bottom",
            yref="y"
        ),
        dict(
            x="2014-09-13",
            y=0,
            arrowcolor="rgba(76, 175, 80, 0.1)",
            arrowsize=0.3,
            ax=0,
            ay=30,
            text="state2",
            xref="x",
            yanchor="bottom",
            yref="y"
        )
    ],
)

# # Add shapes
# # 구간 설정(차트의 배경의 색 구분)
# fig.update_layout(
#     shapes=[
#         dict(
#             fillcolor="rgba(63, 81, 181, 0.2)",
#             line={"width": 0},
#             type="rect",
#             x0="2013-01-15",
#             x1="2013-10-17",
#             xref="x",
#             y0=0,
#             y1=0.95,
#             yref="paper"
#         ),
#         dict(
#             fillcolor="rgba(76, 175, 80, 0.1)",
#             line={"width": 0},
#             type="rect",
#             x0="2013-10-22",
#             x1="2015-08-05",
#             xref="x",
#             y0=0,
#             y1=0.95,
#             yref="paper"
#         )
#     ]
# )

# Update axes
# xaxis 는 아래 부분
# yaxis 는 y 축의 가장 아래 부분
# side = 는 y축 어디에 보이게 할건지
# domain 는 0,1사이 값으로 얼마의 비중으로 보일지
fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"],
        rangeslider=dict(
            autorange=True,
            range=["2012-10-31 18:36:37.3129", "2016-05-10 05:23:22.6871"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 28.4406294707],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.2, 0.4],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100.621222297],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    ),
    yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.4, 0.6],
        linecolor="#795548",
        mirror=True,
        range=[-3.73690396239, 22.2369039624],
        showline=True,
        side="right",
        tickfont={"color": "#795548"},
        tickmode="auto",
        ticks="",
        title="mg/L",
        titlefont={"color": "#795548"},
        type="linear",
        zeroline=False
    ),
    yaxis4=dict(
        anchor="x",
        autorange=True,
        domain=[0.6, 0.8],
        linecolor="#607d8b",
        mirror=True,
        range=[6.63368032236, 8.26631967764],
        showline=True,
        side="right",
        tickfont={"color": "#607d8b"},
        tickmode="auto",
        ticks="",
        title="mmol/L",
        titlefont={"color": "#607d8b"},
        type="linear",
        zeroline=False
    ),
    yaxis5=dict(
        anchor="x",
        autorange=True,
        domain=[0.8, 1],
        linecolor="#2196F3",
        mirror=True,
        range=[-685.336803224, 3718.33680322],
        showline=True,
        side="right",
        tickfont={"color": "#2196F3"},
        tickmode="auto",
        ticks="",
        title="mg/Kg",
        titlefont={"color": "#2196F3"},
        type="linear",
        zeroline=False
    )
)

# Update layout
fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)

fig.show()